In [1]:
!pip install evaluate --quiet
!pip install sacrebleu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pathlib
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset
import evaluate

2025-12-18 17:23:39.254058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766078619.445707      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766078619.500715      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766078619.967966      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766078619.968002      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766078619.968005      55 computation_placer.cc:177] computation placer alr

In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


I0000 00:00:1766078645.113523      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766078645.117425      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Number of devices: 2


In [4]:
config = {
    "max_length": 50,
    "batch_size": 64,
    "split_ratio": 0.9,
    "model_name": "Helsinki-NLP/opus-mt-en-es",
    "epochs": 2,
    "learning_rate": 2e-5,
    "num_samples": 10000,
    "warmup_steps": 500,
}

In [5]:
import urllib.request
import zipfile

url = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
zip_path = "spa-eng.zip"

urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(".")

In [6]:
def load_data(path, num_samples=None):
    text = path.read_text(encoding='utf-8')
    lines = text.splitlines()

    # Limit samples for faster training
    if num_samples:
        lines = lines[:num_samples]

    pairs = [line.split('\t') for line in lines]

    source = [eng for eng, spa in pairs]
    target = [spa for eng, spa in pairs]

    return source, target

In [7]:
path_to_file = pathlib.Path("spa-eng") / "spa.txt"
source_texts, target_texts = load_data(path_to_file, None)


In [8]:
split_idx = int(config["split_ratio"] * len(source_texts))
train_source = source_texts[:split_idx]
train_target = target_texts[:split_idx]
val_source = source_texts[split_idx:]
val_target = target_texts[split_idx:]

In [9]:
print(f"Training samples: {len(train_source)}")
print(f"Validation samples: {len(val_source)}")

Training samples: 107067
Validation samples: 11897


In [10]:
with strategy.scope():
    print("\nLoading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
    model = TFAutoModelForSeq2SeqLM.from_pretrained(config["model_name"])


Loading model and tokenizer...


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tf_model.h5:   0%|          | 0.00/313M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-es.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:
print("\nCreating datasets...")
train_dict = {"source": train_source, "target": train_target}
val_dict = {"source": val_source, "target": val_target}


Creating datasets...


In [12]:
train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

In [13]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples["source"],
        max_length=config["max_length"],
        truncation=True,
        padding="max_length"
    )

    targets = tokenizer(
        examples["target"],
        max_length=config["max_length"],
        truncation=True,
        padding="max_length"
    )

    labels = []
    for label_ids in targets["input_ids"]:
        label_ids_copy = label_ids.copy()
        label_ids_copy = [-100 if token_id == tokenizer.pad_token_id else token_id
                         for token_id in label_ids_copy]
        labels.append(label_ids_copy)

    inputs["labels"] = labels
    return inputs

In [14]:
print("Tokenizing datasets...")
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["source", "target"]
)
val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["source", "target"]
)

Tokenizing datasets...


Map:   0%|          | 0/107067 [00:00<?, ? examples/s]

Map:   0%|          | 0/11897 [00:00<?, ? examples/s]

In [15]:
print("Inspecting first batch of tokenized data:")
for batch in train_dataset.take(1):
    input_ids_batch = np.array(batch['input_ids'])
    print("Shape of input_ids batch:", input_ids_batch.shape)
    print("Tokens for the first example in the batch:")
    print(tokenizer.convert_ids_to_tokens(input_ids_batch))


Inspecting first batch of tokenized data:
Shape of input_ids batch: (50,)
Tokens for the first example in the batch:
['▁Go', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    return_tensors="tf"
)

train_dataset = train_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=True,
    batch_size= config['batch_size'] * strategy.num_replicas_in_sync,
    collate_fn=data_collator
).prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'labels'],
    shuffle=False,
    batch_size= config['batch_size'] * strategy.num_replicas_in_sync,
    collate_fn=data_collator
).prefetch(tf.data.AUTOTUNE)

In [17]:
class WarmupSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, warmup_steps):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)

        warmup_lr = self.initial_learning_rate * (step / warmup_steps)
        decay_lr = self.initial_learning_rate

        return tf.cond(
            step < warmup_steps,
            lambda: warmup_lr,
            lambda: decay_lr
        )
    
    def get_config(self):
        return {
            'initial_learning_rate': self.initial_learning_rate,
            'warmup_steps': self.warmup_steps
        }

In [18]:
with strategy.scope():
    lr_schedule = WarmupSchedule(
    initial_learning_rate=config["learning_rate"],
    warmup_steps=config["warmup_steps"]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer)
    

In [19]:
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True
    )
]

# Train
print("\n" + "="*60)
print("Starting training...")
print("="*60)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=config["epochs"],
    callbacks=callbacks_list,
    verbose=1
)


Starting training...
Epoch 1/2
INFO:tensorflow:Collective all_reduce tensors: 253 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 253 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce IndexedSlices: 2 all_reduces, num_devices =2, group_size = 2, implementation = CommunicationImplementation.NCCL


INFO:tensorflow:Collective all_reduce IndexedSlices: 2 all_reduces, num_devices =2, group_size = 2, implementation = CommunicationImplementation.NCCL


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


837/837 [==============================] - 978s 1s/step - loss: 1.8344 - val_loss: 0.9783 - lr: 2.0000e-05
Epoch 2/2
837/837 [==============================] - 858s 1s/step - loss: 0.7353 - val_loss: 0.7369 - lr: 2.0000e-05


In [20]:
print("\n" + "="*60)
print("Evaluating with BLEU score...")
print("="*60)

metric = evaluate.load("sacrebleu")


Evaluating with BLEU score...


In [21]:
import shutil

In [22]:
# model.save("Transformer-ENG2Span", save_format='tf') 
model.save_pretrained("./my-model")
tokenizer.save_pretrained("./my-model")
# Zip the model directory
shutil.make_archive('my-model', 'zip', './', 'my-model')
print("✓ Model zipped as: my-model.zip")


/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:461: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}
  warnings.warn(


✓ Model zipped as: my-model.zip


In [23]:
# loaded_model = tf.keras.models.load_model("/kaggle/working/Transformer-ENG2Span",compile=False)
loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/my-model")
loaded_tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/my-model")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /kaggle/working/my-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [24]:
def translate(text,tokenizer,model, max_length=50):
    """Translate English text to Spanish"""
    inputs = tokenizer(text, return_tensors="tf", max_length=max_length, truncation=True)
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

In [25]:
print("\n" + "="*50)
print("Sample Translations:")
print("="*50)

for i in range(5):
    idx = np.random.randint(0, len(val_source))
    input_text = val_source[idx]
    expected = val_target[idx]
    predicted = translate(input_text,loaded_tokenizer,loaded_model)

    print(f"\nInput:     {input_text}")
    print(f"Expected:  {expected}")
    print(f"Predicted: {predicted}")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.



Sample Translations:

Input:     It is important to remember who your friends are.
Expected:  Es importante recordar quiénes son tus amigos.
Predicted: Es importante recordar quiénes son tus amigos.

Input:     "Will you have some more coffee?" "No, thanks. I've had enough."
Expected:  "¿Se va a servir un poco más de café?" "No gracias. Ya he bebido suficiente."
Predicted: "¿Tomarás un poco más de café?" "No, gracias. Ya he tenido suficiente."

Input:     They looked at the photo taken of me when I was a boy and laughed.
Expected:  Cuando vieron mi foto de cuando era pequeño se burlaron.
Predicted: Ellos miraron la foto que me hicieron cuando era un niño y se rieron.

Input:     You don't have to talk about it if you don't want to.
Expected:  No tenéis que hablar de eso si no queréis.
Predicted: No tienes que hablar de eso si no quieres.

Input:     He dedicated his whole life to helping poor people.
Expected:  Él dedicó toda su vida a ayudar a la gente pobre.
Predicted: Dedicó toda s

In [ ]:
predictions = []
references = []

print("Generating translations for validation set...")
for i in range(len(val_source)):
    pred = translate(val_source[i],loaded_model)
    predictions.append(pred)
    references.append([val_target[i]])  # BLEU expects list of references

bleu_score = metric.compute(predictions=predictions, references=references)
print(f"\nValidation BLEU Score: {bleu_score['score']:.2f}")

In [ ]:
model.save_weights("model_weights.h5")

In [ ]:
import pickle

weights = model.get_weights()
with open("model_weights.pkl", "wb") as f:
    pickle.dump(weights, f)